<a href="https://colab.research.google.com/github/jyjoon001/CSW4020/blob/main/CSW4020_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#정보처리및자연어처리 3차과제
> 20161482 박준용

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CSW4020

Mounted at /content/drive
/content/drive/MyDrive/CSW4020


In [33]:
!pip install konlpy
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader
import re
import os
import konlpy
import random
from konlpy.tag import Kkma
kkma = Kkma()

##개요

데이터파일 ```entity_data_utf8.txt```를 대상으로 ```NaiveBayesClassifier```를 이용하여 accuracy가 높은 분류 모형 만들기

In [4]:
a = open('./entity_data_utf8.txt', encoding='utf-8').read().split('\n1\t')
b = []
b.append(a[0]+'\n')
for i in range(1, len(a)):
    b.append('1\t'+a[i]+'\n')
b[:5]

['1\t비토리오\tPER\n2\t양일\tDAT\n3\t만에\t-\n4\t영사관\tORG\n5\t감호\tCVL\n6\t용퇴,\t-\n7\t항룡\t-\n8\t압력설\t-\n9\t의심만\t-\n10\t가율\t-\n',
 '1\t이\t-\n2\t음경동맥의\t-\n3\t직경이\t-\n4\t8\tNUM\n5\t19mm입니다\tNUM\n6\t.\t-\n',
 '1\t9세이브로\tNUM\n2\t구완\t-\n3\t30위인\tNUM\n4\tLG\tORG\n5\t박찬형은\tPER\n6\t평균자책점이\t-\n7\t16.45로\tNUM\n8\t준수한\t-\n9\t편이지만\t-\n10\t22⅓이닝\tNUM\n11\t동안\t-\n12\t피홈런이\t-\n13\t31개나\tNUM\n14\t된다\t-\n15\t.\t-\n',
 '1\t7승\tNUM\n2\t25패는\tNUM\n3\t상트페테르부르크가\tLOC\n4\t역대\t-\n5\t월드리그에\tEVT\n6\t출진한\t-\n7\t분별\t-\n8\t최선의\t-\n9\t성적이다\t-\n10\t.\t-\n',
 '1\t▲\t-\n2\t퍼거슨\tPER\n3\t씨족의\tCVL\n4\t꾀\t-\n']

In [5]:
len(b)

90000

In [6]:
result = []
for j in range(0, len(b)):
    temp1 = re.findall('\t(.*)\t',b[j])
    temp2 = re.findall('.*\t(.*)\n',b[j])
    temp = []
    for i in range(0, len(temp1)):
        temp.append((temp1[i], temp2[i]))
    result.append(temp)

In [7]:
flatted = [i for j in result for i in j]
random.shuffle(flatted)

In [9]:
size = int(len(flatted) *0.9)
train_words, test_words = flatted[:size], flatted[size:]

1. n-gram

In [8]:
def ngram_features(name):
    features = {}
    features["last_two_letters"] = name[-2:]
    features["last_letter"] = name[-1:]
    return features 

In [10]:
train_set = [(ngram_features(n), gender) for (n, gender) in train_words]
test_set = [(ngram_features(n), gender) for (n, gender) in test_words]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.7938189886985464

In [11]:
classifier.show_most_informative_features(20)

Most Informative Features
        last_two_letters = '상오'              TIM : -      =   5786.4 : 1.0
        last_two_letters = '1일'              DAT : -      =   5471.3 : 1.0
        last_two_letters = '.'                 - : NUM    =   5301.7 : 1.0
             last_letter = '%'               NUM : -      =   4034.6 : 1.0
        last_two_letters = '림픽'              EVT : -      =   3429.6 : 1.0
        last_two_letters = '더파'              NUM : -      =   3138.0 : 1.0
        last_two_letters = '0년'              DAT : -      =   3045.3 : 1.0
        last_two_letters = '6일'              DAT : -      =   2942.8 : 1.0
        last_two_letters = '4년'              DAT : -      =   2857.4 : 1.0
        last_two_letters = '8일'              DAT : -      =   2806.1 : 1.0
        last_two_letters = '9일'              DAT : -      =   2780.5 : 1.0
        last_two_letters = '3일'              DAT : -      =   2746.3 : 1.0
        last_two_letters = '1월'              DAT : -      =   2293.6 : 1.0

2. kormorph

In [16]:
def kormorph_features(name):
    features = {}
    features["kor_morphs"] = kkma.pos(name)[0][1]
    return features 

In [17]:
train_set = [(kormorph_features(n), gender) for (n, gender) in train_words]
test_set = [(kormorph_features(n), gender) for (n, gender) in test_words]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.7309182195979615

In [19]:
classifier.show_most_informative_features(20)

Most Informative Features
              kor_morphs = 'SF'                - : CVL    =   3887.2 : 1.0
              kor_morphs = 'ECE'             LOC : -      =    588.3 : 1.0
              kor_morphs = 'NR'              NUM : -      =    437.3 : 1.0
              kor_morphs = 'NNP'             LOC : DAT    =    437.2 : 1.0
              kor_morphs = 'UN'              PER : NUM    =    374.6 : 1.0
              kor_morphs = 'VXV'               - : LOC    =    195.1 : 1.0
              kor_morphs = 'ECD'             LOC : -      =    136.1 : 1.0
              kor_morphs = 'OL'              TRM : PER    =    117.9 : 1.0
              kor_morphs = 'XPN'             PER : NUM    =    115.8 : 1.0
              kor_morphs = 'VCP'             LOC : NUM    =     85.3 : 1.0
              kor_morphs = 'XR'                - : NUM    =     83.6 : 1.0
              kor_morphs = 'XSV'             AFW : -      =     71.6 : 1.0
              kor_morphs = 'NNB'               - : TIM    =     70.3 : 1.0

3. hangul-jaso

In [21]:
def jaso1(a):
    if re.match('[가-힣]', a):
        initial = ['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
        midial = ['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ']
        final = ['None', 'ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
        Chr_ord = ord(a) -44032
        ini_index = Chr_ord // (21*28)
        mid_index = (Chr_ord // 28)% 21
        final_index = Chr_ord % 28
        return [initial[ini_index], midial[mid_index], final[final_index]]
    else:
        return ['None', 'None', 'None']

In [22]:
def jaso_features(name):
    features = {}
    features["ini_index"] = jaso1(name[0][-1:])[0]
    features["mid_index"] = jaso1(name[0][-1:])[1]
    features["final_index"] = jaso1(name[0][-1:])[2] 
    return features 

In [23]:
jaso1(flatted[2][0][-1:])[2]

'None'

In [24]:
train_set = [(jaso_features(n), gender) for (n, gender) in train_words]
test_set = [(jaso_features(n), gender) for (n, gender) in test_words]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.6238458790123921

In [26]:
classifier.show_most_informative_features(20)

Most Informative Features
             final_index = 'ㅊ'               PLT : ORG    =   1338.5 : 1.0
               ini_index = 'ㄲ'               PLT : DAT    =   1237.4 : 1.0
             final_index = 'ㅆ'                 - : CVL    =    754.6 : 1.0
               ini_index = 'ㅋ'               FLD : DAT    =    652.7 : 1.0
               ini_index = 'ㅃ'               PLT : NUM    =    488.9 : 1.0
             final_index = 'ㄶ'                 - : CVL    =    296.3 : 1.0
               mid_index = 'ㅡ'               FLD : TIM    =    251.4 : 1.0
             final_index = 'ㅈ'               TIM : PER    =    224.6 : 1.0
               ini_index = 'ㄹ'               PER : DAT    =    211.1 : 1.0
               ini_index = 'ㄸ'               PLT : NUM    =    202.6 : 1.0
               ini_index = 'ㅌ'               MAT : DAT    =    199.8 : 1.0
               mid_index = 'ㅙ'               LOC : NUM    =    149.4 : 1.0
               mid_index = 'ㅔ'               FLD : TIM    =    117.4 : 1.0

4. length features

In [27]:
def len_features(name):
    letters = ''.join(name)
    features = {}
    features['length'] = len(name)
    features["ratio_alphabet"] =  len(re.findall('[가-힣]', letters, re.I))/len(letters)
    features["ratio_digit"] = len(re.findall('\d', letters))/len(letters)
    features["ratio_punct"] = len(re.findall('[^\w]', letters))/len(letters)
    return features 

In [28]:
train_set = [(len_features(n), gender) for (n, gender) in train_words]
test_set = [(len_features(n), gender) for (n, gender) in test_words]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.7282761992515843

In [29]:
classifier.show_most_informative_features(20)

Most Informative Features
          ratio_alphabet = 0.13636363636363635    TRM : -      =   7311.1 : 1.0
                  length = 19                TRM : -      =   5111.6 : 1.0
             ratio_punct = 0.13636363636363635    TRM : -      =   4739.0 : 1.0
             ratio_punct = 0.15              TRM : -      =   4712.6 : 1.0
             ratio_digit = 0.5714285714285714    NUM : -      =   4472.7 : 1.0
                  length = 22                TRM : -      =   3322.3 : 1.0
             ratio_digit = 0.6666666666666666    NUM : PER    =   3193.3 : 1.0
          ratio_alphabet = 0.09523809523809523    TRM : -      =   2882.8 : 1.0
          ratio_alphabet = 0.26666666666666666    MAT : -      =   2660.7 : 1.0
                  length = 18                TRM : -      =   2605.0 : 1.0
             ratio_punct = 0.19047619047619047    TRM : -      =   2524.4 : 1.0
                  length = 21                TRM : -      =   2326.9 : 1.0
             ratio_punct = 0.066666666666

5. best features

In [30]:
def best_features(name):
    letters = ''.join(name)
    features = {}
    features["first_letter"] = name[0]
    features["last_two_letters"] = name[-2:]
    features["last_three_letters"] = name[-3:]
    features["ratio_digit"] = len(re.findall('\d', letters))/len(letters)
    features['length'] = len(name)
    return features 

In [31]:
train_set = [(best_features(n), gender) for (n, gender) in train_words]
test_set = [(best_features(n), gender) for (n, gender) in test_words]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.8633577163918088

In [32]:
classifier.show_most_informative_features(20)

Most Informative Features
      last_three_letters = '.'                 - : NUM    =   8806.8 : 1.0
        last_two_letters = '상오'              TIM : -      =   5786.4 : 1.0
        last_two_letters = '1일'              DAT : -      =   5471.3 : 1.0
        last_two_letters = '.'                 - : NUM    =   5301.7 : 1.0
                  length = 19                TRM : -      =   5111.6 : 1.0
             ratio_digit = 0.5714285714285714    NUM : -      =   4472.7 : 1.0
            first_letter = '1'               NUM : PER    =   4375.1 : 1.0
            first_letter = '7'               DAT : -      =   4175.7 : 1.0
            first_letter = '2'               NUM : -      =   4037.0 : 1.0
            first_letter = '.'                 - : CVL    =   3774.2 : 1.0
        last_two_letters = '림픽'              EVT : -      =   3429.6 : 1.0
                  length = 22                TRM : -      =   3322.3 : 1.0
             ratio_digit = 0.6666666666666666    NUM : PER    =   3193